Following Chris' advice, I added hyper-diffusion to the SAM model using Matt Wyantt's code. After some trial and error, I settled upon a diffusion parameter of 

$$ k = .5  U \Delta x^3,$$ 
where $U$ is the typical velocity scale of 50 m/s. This is chosen so that
$Re = (L^3 U)/ k \approx 1$ when $L$ is the grid-scale $\Delta x$.

In [ ]:
%matplotlib inline

In [ ]:
ds = xr.open_mfdataset("../data/runs/tmp/OUT_2D/*.nc")

In [ ]:
ds.V200[::12].plot(col='time', col_wrap=3)

In [ ]:
ds.U850[::12].plot(col='time', col_wrap=3)

The velocity field are markedly smoother.

In [ ]:
ds.PW[::12].plot(col='time', col_wrap=3)

this might be too much diffusion. Jablownowski suggest the CAM default hyper-diffusivity is around 1e15 for 160km resolutions.

In [ ]:
ds = xr.open_mfdataset("../data/runs/khyp_1e15/OUT_2D/*.nc")

In [ ]:
ds.V200[::12].plot(col='time', col_wrap=3)

In [ ]:
ds.V850[::12].plot(col='time', col_wrap=3)

Convert the SAM output

!docker run  -w $(pwd) -v /Users:/Users nbren12/uwnet 2Dbin2nc ../khyp_1e16/OUT_2D/*.2Dbin >/dev/null

Open it

In [ ]:
ds = xr.open_mfdataset("../data/runs/khyp_1e16/OUT_2D/*.nc")

In [ ]:
ds.V200[::12].plot(col='time', col_wrap=3)

In [ ]:
ds.V850[::12].plot(col='time', col_wrap=3)

I was also able to use a 100 second time step with this level of diffusion. Unfortunately, there is still visible evidence of gibbs phenomena in these solutions. See the strong patch of northerly flow near x= 1.5e7m in the first three panels. For now, this seem like a decent value of the hyper diffusivity. Maybe I should smooth the initial conditions somewhat as well.

Let's find a trained neural network to use

In [ ]:
import pymongo
from bson.objectid import ObjectId


db = pymongo.MongoClient()
runs = db.uwnet.run

In [ ]:
import re

In [ ]:
runs_sorted = runs\
.find({"args.input": {"$not": re.compile('subset\.nc')}, 'config.outputs': {"$ne": ('U', 28)}})\
.sort([('start_timestamp', pymongo.DESCENDING)])

In [ ]:
df = pd.DataFrame(list(runs_sorted))

In [ ]:
df['n_epochs'] = df.args.apply(lambda x: x['n_epochs'])

In [ ]:
print(df[['_id', 'n_epochs', 'start_timestamp']].head())

In [ ]:
run_id = ObjectId('5b92ab0fe6060eef257d450a')
# run_id = ObjectId('5b92d2bbe6060eb1f80f945e')

epoch = db.uwnet.epoch.find_one({'run_id': run_id, 'epoch': 2})
run = db.uwnet.run.find_one({'_id': run_id})

In [ ]:
import os

In [ ]:
model_dir_name = os.path.dirname(epoch['model'])
# model_dir_name = ''
column_file = f"{model_dir_name}/cols.nc"

if not os.path.exists(column_file):
    %run -m uwnet.columns {epoch['model']} ../data/subset.nc $column_file

cols_ds = xr.open_dataset(column_file)

In [ ]:
loc = cols_ds.isel(x=0, y=0)

In [ ]:
loc.QN.plot(x='time')

In [ ]:
loc.QT.plot(x='time')

There is some problem with the humidity field near z=7500 m.

# NN + SAM + Hyperdiffusion

Let's now run this neural network inside SAM

In [ ]:
%%script env model={epoch['model']} model_dir_name={model_dir_name} assets=../assets  bash 

sam_dir_name=$model_dir_name/SAM

# make running directory
if [[ -d $sam_dir_name ]]
then
    echo $sam_dir_name "already exists. Stopping execution"
    exit 1
fi

mkdir $sam_dir_name

echo "Running SAM in $sam_dir_name"

# setup necessary files for running SAM
cp -r $assets/NG1 $sam_dir_name

# run same
(cd $sam_dir_name
 /Users/noah/workspace/research/uwnet/bin/run_sam_nn_docker.sh $model > out 2> err
)

Convert and load the two dimensional output data.

In [ ]:
output_2d_dir = f'{model_dir_name}/SAM/OUT_2D'
output_3d_dir = f'{model_dir_name}/SAM/OUT_3D'

ds2d = xr.open_mfdataset(f"{output_2d_dir}/*.nc")
ds3d_nn = xr.open_mfdataset(f"{output_3d_dir}/*.nc")

# add anomaly field
anom_nn = ds3d_nn - ds3d_nn.mean(['x', 'time'])

What does the preicpitation field look like?

In [ ]:
ds2d.PW[::12].plot(col='time')

In [ ]:
ds2d.Prec[::12].plot(col='time')

There are some grid point storms in this simulation. What does the vertical structure of this instability look like?

In [ ]:
anom_nn.QT.isel(y=32)[::12].plot(x='x', y='z', col='time')

There is an excessively moist column!

In [ ]:
anom_nn.W.isel(y=32)[::12].plot(x='x', y='z', col='time')

Which has a large vertical velocity.

In [ ]:
anom_nn.SLI.isel(y=32)[::12].plot(x='x', y='z', col='time')

The aren't any large anomalies in the temperature field. 

In [ ]:
ds_ng = xr.open_mfdataset("/Users/noah/Data/2018-05-30-NG_5120x2560x34_4km_10s_QOBS_EQX/coarse/2d/all.nc").sortby('time')

In [ ]:
ds_ng.Prec.mean(['x' ,'y'])[:100].plot()

In [ ]:
ds_ng.PW.mean(['x' ,'y'])[:100].plot()

## Model trained for 9 epochs

In [ ]:
path = '/Users/noah/workspace/research/uwnet/trained_models/5b/92d2bbe6060eb1f80f945e/SAM/OUT_2D'
file = 'NG1_test_1.2Dbin'
# !docker run -v /Users:/Users -w {path} nbren12/uwnet 2Dbin2nc {file} > /dev/null

In [ ]:
ds_9epoch = xr.open_mfdataset(os.path.join(path, '*.nc'))

In [ ]:
ds_9epoch.Prec[::12].plot(col='time', vmax=100)

In [ ]:
ds_9epoch.PW[::12].plot(col='time', vmax=100)

The problem is still here. Also there seems to be a strong latitudinal dependence with horizontal striations. Is this caused by using SOLIN and SST as inputs. What happens if I remove them.

In [ ]:
ds_9epoch.PW.mean(['x', 'y']).plot()

the precipitable water is increasing way too much in this simulation.

In [ ]:
ds_9epoch.Prec.mean(['x', 'y']).plot()

The diurnal cycle of precip is all wrong.

In [ ]:
ds_9epoch.PW[-1].mean(['x']).plot()
ds_ng.PW.sel(time=103.625).mean('x').plot()

In [ ]:
ds_9epoch.VSFC[::12].plot(col='time',  col_wrap=3, vmax=40)

In [ ]:
ds_ng.VSFC[:12*6:12].plot(col='time',  col_wrap=3, vmax=40)

In [ ]:
ds_9epoch.VSFC.mean('x').plot(y='y')

In [ ]:
ds_9epoch.W500.mean('x').plot(y='y', vmax=.01)

In [ ]:
ds_9epoch.W500[::6].plot(col='time', col_wrap=5, vmax=.2)

Why are the zonally averaged meridional winds so drastically altered.

In [ ]:
ds_ng.VSFC.mean('x').sel(time=slice(None, 103.5)).plot(y='y')

In [ ]:
ds_ng.W500.mean('x').sel(time=slice(None, 103.5)).plot(y='y')

Does this also happen when the neural network physics aren't on?

In [ ]:
ds_no = xr.open_mfdataset("../data/runs/khyp_1e16/OUT_2D/NG1_test_1.2Dbin_1.nc")

In [ ]:
ds_no.VSFC.mean('x').plot(y='y')

In [ ]:
ds_no.W500.mean('x').plot(y='y', vmax=.01)

The vertical velocity on the equator seems to disappear very quickly.

In [ ]:
ds_9epoch.Prec.mean('x').plot(y='y', vmax=18)

In [ ]:
ds_ng.Prec.mean('x').sel(time=slice(None, 103.5)).plot(y='y', vmax=18)

# Try removing SOLIN and SST as inputs

This might help with horizontal striations in this data.


`trained_models/5b/92fc43e6060e6a470454ec`

In [ ]:
path = '/Users/noah/workspace/research/uwnet/trained_models/5b/92fc43e6060e6a470454ec/SAM/OUT_2D'
file = 'NG1_test_1.2Dbin'
!docker run -v /Users:/Users -w {path} nbren12/uwnet 2Dbin2nc {file} > /dev/null

In [ ]:
ds2d_nosolin = xr.open_mfdataset(path + '/*.nc')

In [ ]:
ds2d_nosolin.Prec[::6].plot(col='time', col_wrap=5)

In [ ]:
ds_ng.Prec[:50:6].plot(col='time', col_wrap=5, vmax=100)

In [ ]:
ds2d_nosolin.W500[::6].plot(col='time', col_wrap=5)

In [ ]:
ds2d_nosolin.VSFC.mean('x').plot(y='y')

In [ ]:
ds2d_nosolin.W500.mean('x').plot(y='y', vmax=.01)

It looks like it is happening again, even though the simulation is not finished yet. I need to figure out why the scheme is not able to sustain upward motion in the tropics. It might be easier to start from a zonally homogeneous profile or something, to see what the scheme does. Part of my guess is that the vertical velocity is so speckled in the initial conditions, that the neural network is accustomed to forcings which far exceed what it sees in the model.